In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()


from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy Release
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
df = spark.read.csv('original.csv', header=True)
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [ ]:
df.dtypes

[('id', 'string'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('City', 'string'),
 ('JobTitle', 'string'),
 ('Salary', 'string'),
 ('Latitude', 'string'),
 ('Longitude', 'string')]

In [ ]:
# Update the schema of the dataframe during load

from pyspark.sql.types import *

schema = StructType([
  StructField("id", IntegerType()),
  StructField("first_name", StringType()),
  StructField("last_name", StringType()),
  StructField("gender", StringType()),
  StructField("City", StringType()),
  StructField("JobTitle", StringType()),
  StructField("Salary", StringType()),
  StructField("Latitude", FloatType()),
  StructField("Longitude", FloatType())
  ])

df2 = spark.read.csv('original.csv', header=True, schema=schema)

In [ ]:
df2.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude| Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18| 50.577408| 16.496717|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|  48.82316| 103.52182|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52| 39.994747|116.339775|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23| 44.504723| 38.130016|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.648994|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16| 53.426613|-6.1644998|
|  7|     Masha|    Divers|Female|         Dachun|              

## Inspecting Dataframe

In [ ]:
df2.dtypes

[('id', 'int'),
 ('first_name', 'string'),
 ('last_name', 'string'),
 ('gender', 'string'),
 ('City', 'string'),
 ('JobTitle', 'string'),
 ('Salary', 'string'),
 ('Latitude', 'float'),
 ('Longitude', 'float')]

In [ ]:
df.head(5)

[Row(id='1', first_name='Melinde', last_name='Shilburne', gender='Female', City='Nowa Ruda', JobTitle='Assistant Professor', Salary='$57438.18', Latitude='50.5774075', Longitude='16.4967184'),
 Row(id='2', first_name='Kimberly', last_name='Von Welden', gender='Female', City='Bulgan', JobTitle='Programmer II', Salary='$62846.60', Latitude='48.8231572', Longitude='103.5218199'),
 Row(id='3', first_name='Alvera', last_name='Di Boldi', gender='Female', City=None, JobTitle=None, Salary='$57576.52', Latitude='39.9947462', Longitude='116.3397725'),
 Row(id='4', first_name='Shannon', last_name="O'Griffin", gender='Male', City='Divnomorskoye', JobTitle='Budget/Accounting Analyst II', Salary='$61489.23', Latitude='44.5047212', Longitude='38.1300171'),
 Row(id='5', first_name='Sherwood', last_name='Macieja', gender='Male', City='Mytishchi', JobTitle='VP Sales', Salary='$63863.09', Latitude=None, Longitude='37.6489954')]

In [ ]:
df.first()

Row(id='1', first_name='Melinde', last_name='Shilburne', gender='Female', City='Nowa Ruda', JobTitle='Assistant Professor', Salary='$57438.18', Latitude='50.5774075', Longitude='16.4967184')

In [ ]:
df.describe().show()

+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+-----------------+------------------+
|summary|               id|first_name|last_name|gender|               City|           JobTitle|   Salary|         Latitude|         Longitude|
+-------+-----------------+----------+---------+------+-------------------+-------------------+---------+-----------------+------------------+
|  count|             1000|      1000|     1000|  1000|                999|                998|     1000|              999|              1000|
|   mean|            500.5|      null|     null|  null|               null|               null|     null|25.43151724234234|43.337564614499996|
| stddev|288.8194360957494|      null|     null|  null|               null|               null|     null| 24.5790825486909| 69.42064539970089|
|    min|                1|   Abagail|    Abbay|Female|             Abéché|Account Coordinator|$10101.92|       -0.6256517|        -0.4889547|

In [ ]:
df.columns

['id',
 'first_name',
 'last_name',
 'gender',
 'City',
 'JobTitle',
 'Salary',
 'Latitude',
 'Longitude']

In [ ]:
df.count()

1000

In [ ]:
df.distinct().count()

1000

## Handling Null and duplicates values

In [ ]:
# drop all rows that have NA values
df_dropped = df.na.drop()


In [ ]:
df_dropped.count()

997

In [ ]:
df_null_jobs = df.filter(df.JobTitle.isNotNull()) # drop rows where the job title is null

In [ ]:
df_null_jobs.count()

998

In [ ]:
from pyspark.sql.functions import *

In [ ]:
df_handled = df.withColumn('clean_city',
                           when(df.City.isNull(), 'unknown')
                            .otherwise(df.City))

In [ ]:
df_handled.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|     clean_city|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|      Nowa Ruda|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|         Bulgan|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|        unknown|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|  Divnomorskoye|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|      Mytishchi|
|  6|     Maris|      Folk|Femal

In [ ]:
df_no_duplicates = df.dropDuplicates() # drop duplicates

In [ ]:
df_no_duplicates.count()

1000

## Selecting and filtering Data

In [ ]:
df_select = df.select('first_name', 'last_name')
df_select.show()

+----------+----------+
|first_name| last_name|
+----------+----------+
|   Melinde| Shilburne|
|  Kimberly|Von Welden|
|    Alvera|  Di Boldi|
|   Shannon| O'Griffin|
|  Sherwood|   Macieja|
|     Maris|      Folk|
|     Masha|    Divers|
|   Goddart|     Flear|
|      Roth|O'Cannavan|
|      Bran|   Trahear|
|    Kylynn|   Lockart|
|       Rey|    Meharg|
|      Kerr|    Braden|
|    Mickie| Whanstall|
|    Kaspar|     Pally|
|    Norbie|    Gwyllt|
|    Claude|    Briant|
|     Thain|    Habbon|
|  Tiffanie|  Pattison|
|    Ettore|  Gerriets|
+----------+----------+
only showing top 20 rows



In [ ]:
df_renamed = df.withColumnRenamed('JobTitle', 'job_title')
df_renamed.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|           job_title|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [ ]:
df_filter = df.filter(df.first_name == 'Alvera')
df_filter.show()

+---+----------+---------+------+----+--------+---------+----------+-----------+
| id|first_name|last_name|gender|City|JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+---------+------+----+--------+---------+----------+-----------+
|  3|    Alvera| Di Boldi|Female|null|    null|$57576.52|39.9947462|116.3397725|
+---+----------+---------+------+----+--------+---------+----------+-----------+



In [ ]:
df_filter = df.filter(df.first_name.like("%vera"))
df_filter.show()

+---+----------+---------+------+----+--------+---------+----------+-----------+
| id|first_name|last_name|gender|City|JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+---------+------+----+--------+---------+----------+-----------+
|  3|    Alvera| Di Boldi|Female|null|    null|$57576.52|39.9947462|116.3397725|
+---+----------+---------+------+----+--------+---------+----------+-----------+



In [ ]:
df_filter = df.filter(df.first_name.like("%ver%"))
df_filter.show()

+---+----------+-----------+------+------------+--------------------+---------+----------+-----------+
| id|first_name|  last_name|gender|        City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+-----------+------+------------+--------------------+---------+----------+-----------+
|  3|    Alvera|   Di Boldi|Female|        null|                null|$57576.52|39.9947462|116.3397725|
|179|   Laverna|  Yuryichev|Female|  Lyaskovets| Marketing Assistant|$95106.34|43.0683838| 25.7428451|
|775|   Alverta|   MacNulty|Female|  Megalópoli| Geological Engineer|$17299.62| 37.401244| 22.1364871|
|868|    Denver|      Olman|  Male| Ryazanskaya| Structural Engineer|$56653.70| 44.956561| 39.5853276|
|909|     Avery|Tunniclisse|  Male|Karlovy Vary|          Pharmacist|$46071.13|50.2318521| 12.8719616|
|923|    Traver|  Stillwell|  Male|      Perico|Occupational Ther...|$79520.50|22.7686569|-81.0174917|
+---+----------+-----------+------+------------+--------------------+----

In [ ]:
df_filter = df.filter(df.first_name.like("%din"))
df_filter.show()

+---+----------+-------------+------+-----------+--------+---------+-----------+----------+
| id|first_name|    last_name|gender|       City|JobTitle|   Salary|   Latitude| Longitude|
+---+----------+-------------+------+-----------+--------+---------+-----------+----------+
|901|     Aldin|Matuszkiewicz|  Male|East London|Operator|$41468.83|-32.9549324|27.9319131|
+---+----------+-------------+------+-----------+--------+---------+-----------+----------+



In [ ]:
df_filter = df.filter(df.first_name.endswith("ael"))
df_filter.show()

+---+----------+---------+------+-----------------+---------------+---------+----------+----------+
| id|first_name|last_name|gender|             City|       JobTitle|   Salary|  Latitude| Longitude|
+---+----------+---------+------+-----------------+---------------+---------+----------+----------+
| 84|      Dael|   Bediss|  Male|Chaloem Phra Kiat| Civil Engineer|$54840.20| 9.9533828|98.6111657|
|393|   Michael|   O'Quin|  Male|        Doloplazy|Cost Accountant|$42590.49|49.5671764|17.4114814|
+---+----------+---------+------+-----------------+---------------+---------+----------+----------+



In [ ]:
df_filter = df.filter(df.first_name.startswith("Ale"))
df_filter.show()

+---+----------+---------+------+--------------+--------------------+---------+----------+-----------+
| id|first_name|last_name|gender|          City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+---------+------+--------------+--------------------+---------+----------+-----------+
|101|     Alene|      Odd|Female| María la Baja|Accounting Assist...|$37379.03| 9.9141597|-75.4111644|
|287|    Alexei|  Harriot|  Male|     Shangjing|    Junior Executive|$55711.38| 24.390595| 114.522393|
|429|Alessandro|  Shearer|  Male|Banjar Kelodan|            VP Sales|$51979.53|-8.1233399|115.3389332|
|947|    Alexei|  Durston|  Male|         Zhuxi|     Cost Accountant|$15849.42| 32.318255| 109.715304|
+---+----------+---------+------+--------------+--------------------+---------+----------+-----------+



In [ ]:
df_filter = df.filter(df.id.between(1,6))
df_filter.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
+---+----------+----------+------+---------------+-----

In [ ]:
df_filter = df.filter(df.first_name.isin("Maris", "Alvera"))
df_filter.show()

+---+----------+---------+------+---------------+--------------+---------+----------+-----------+
| id|first_name|last_name|gender|           City|      JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+---------+------+---------------+--------------+---------+----------+-----------+
|  3|    Alvera| Di Boldi|Female|           null|          null|$57576.52|39.9947462|116.3397725|
|  6|     Maris|     Folk|Female|Kinsealy-Drinan|Civil Engineer|$30101.16|53.4266145| -6.1644997|
+---+----------+---------+------+---------------+--------------+---------+----------+-----------+



In [ ]:
df_substr = df.select(df.first_name, df.first_name.substr(1,5).alias("short_name"))

In [ ]:
df_substr.show()

+----------+----------+
|first_name|short_name|
+----------+----------+
|   Melinde|     Melin|
|  Kimberly|     Kimbe|
|    Alvera|     Alver|
|   Shannon|     Shann|
|  Sherwood|     Sherw|
|     Maris|     Maris|
|     Masha|     Masha|
|   Goddart|     Godda|
|      Roth|      Roth|
|      Bran|      Bran|
|    Kylynn|     Kylyn|
|       Rey|       Rey|
|      Kerr|      Kerr|
|    Mickie|     Micki|
|    Kaspar|     Kaspa|
|    Norbie|     Norbi|
|    Claude|     Claud|
|     Thain|     Thain|
|  Tiffanie|     Tiffa|
|    Ettore|     Ettor|
+----------+----------+
only showing top 20 rows



## Applying Multiple filters

In [ ]:
df_filter = df.filter(df.first_name.isin("Tiffanie", "Alvera") & (df.JobTitle.like("%Assistant%")))

In [ ]:
df_filter.show()

+---+----------+---------+------+----+--------+------+--------+---------+
| id|first_name|last_name|gender|City|JobTitle|Salary|Latitude|Longitude|
+---+----------+---------+------+----+--------+------+--------+---------+
+---+----------+---------+------+----+--------+------+--------+---------+



In [ ]:
df_filter = df.filter(df.first_name.isin("Tiffanie", "Alvera") | (df.JobTitle.like("%Assistant%")))

In [ ]:
df_filter.show()

+---+----------+---------+------+-------------+--------------------+---------+----------+-----------+
| id|first_name|last_name|gender|         City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+---------+------+-------------+--------------------+---------+----------+-----------+
|  1|   Melinde|Shilburne|Female|    Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  3|    Alvera| Di Boldi|Female|         null|                null|$57576.52|39.9947462|116.3397725|
| 14|    Mickie|Whanstall|  Male|  Springfield|Assistant Media P...|$50838.53|42.1014803|-72.5766759|
| 17|    Claude|   Briant|Female|    Mieścisko|Research Assistan...|$51862.48|52.7441662| 17.3278637|
| 19|  Tiffanie| Pattison|Female|  Jabungsisir|Senior Financial ...|$91925.08|-7.7232567|113.4686802|
| 25|     Shaun|   Bridle|Female|        Xitou| Office Assistant IV|$71279.60| 29.866917| 118.403786|
| 39|    Valida| Salzberg|Female|  Pangnirtung| Assistant Professor|$94224.48|  66

## Running SQL on Dataframes

In [ ]:
new_df = df.registerTempTable('original')

In [ ]:
query1 = spark.sql("select * from original")

In [ ]:
query1.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      Civil Engineer|$30101.16|53.4266145| -6.1644997|
|  7|     Masha|    Divers|Female|         Dachun|     

In [ ]:
query2 = spark.sql('select concat(first_name, " ", last_name) as full_name from original where gender="Female" ')
query2.show()

+-------------------+
|          full_name|
+-------------------+
|  Melinde Shilburne|
|Kimberly Von Welden|
|    Alvera Di Boldi|
|         Maris Folk|
|       Masha Divers|
|     Kylynn Lockart|
|         Rey Meharg|
|      Claude Briant|
|  Tiffanie Pattison|
|    Lurleen Janczak|
|      Nichol Holtum|
|       Shaun Bridle|
|     Leandra Anfrey|
|    Jaquelyn Hazard|
|  Prudence Honacker|
|       Cherey Liger|
|          Neda Krop|
|    Barbi Fattorini|
|   Lonnie Townshend|
|    Valida Salzberg|
+-------------------+
only showing top 20 rows



## Adding Calculated Columns

In [ ]:
df = df.withColumn('clean_salary', df.Salary.substr(2,100).cast('float'))
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|clean_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|    57438.18|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|     62846.6|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|    57576.52|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|    61489.23|
|  5|  Sherwood|   Macieja|  Male|      Mytishchi|            VP Sales|$63863.09|      null| 37.6489954|    63863.09|
|  6|     Maris|      Folk|Female|Kinsealy-Drinan|      

In [ ]:
df = df.withColumn('monthly_salary', df.clean_salary/12)
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+------------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|clean_salary|    monthly_salary|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+------------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|    57438.18| 4786.514973958333|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|     62846.6|    5237.216796875|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|    57576.52| 4798.043294270833|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|44.5047212| 38.1300171|    61489.23|   5124.1025390625|
|  5|  Sherwood|   Macieja|  Male|      M

In [ ]:
df = df.withColumn('are_they_female', when(df.gender == 'Female', 'Yes').otherwise('No'))
df.show()

+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+------------------+---------------+
| id|first_name| last_name|gender|           City|            JobTitle|   Salary|  Latitude|  Longitude|clean_salary|    monthly_salary|are_they_female|
+---+----------+----------+------+---------------+--------------------+---------+----------+-----------+------------+------------------+---------------+
|  1|   Melinde| Shilburne|Female|      Nowa Ruda| Assistant Professor|$57438.18|50.5774075| 16.4967184|    57438.18| 4786.514973958333|            Yes|
|  2|  Kimberly|Von Welden|Female|         Bulgan|       Programmer II|$62846.60|48.8231572|103.5218199|     62846.6|    5237.216796875|            Yes|
|  3|    Alvera|  Di Boldi|Female|           null|                null|$57576.52|39.9947462|116.3397725|    57576.52| 4798.043294270833|            Yes|
|  4|   Shannon| O'Griffin|  Male|  Divnomorskoye|Budget/Accounting...|$61489.23|4

## Group By and Aggregation

In [ ]:
df1 = df.groupBy('gender').agg(sum('clean_salary'))
df1.show()

+------+--------------------+
|gender|   sum(clean_salary)|
+------+--------------------+
|Female|2.7364519950195312E7|
|  Male|2.8123435678710938E7|
+------+--------------------+



In [ ]:
df2 = df.groupBy('gender').agg(sum('clean_salary').alias('total_salary'),
                               avg('clean_salary').alias('avg_salary'),
                               min('clean_salary').alias('min_salary'),
                               max('clean_salary').alias('max_salary'),
                               stddev('clean_salary').alias('std_salary'))

df2.show()

+------+--------------------+-----------------+----------+----------+------------------+
|gender|        total_salary|       avg_salary|min_salary|max_salary|        std_salary|
+------+--------------------+-----------------+----------+----------+------------------+
|Female|2.7364519950195312E7|55618.94298820185|  10616.44|  99948.28| 26606.13096995577|
|  Male|2.8123435678710938E7|55361.09385573019|  10101.92|  99942.92|25132.221319991764|
+------+--------------------+-----------------+----------+----------+------------------+



In [ ]:
df3 = df.groupBy('gender', 'City').agg(sum('clean_salary').alias('total_salary'),
                               avg('clean_salary').alias('avg_salary'),
                               min('clean_salary').alias('min_salary'),
                               max('clean_salary').alias('max_salary'))

df3.show()

+------+-----------------+----------------+----------------+----------+----------+
|gender|             City|    total_salary|      avg_salary|min_salary|max_salary|
+------+-----------------+----------------+----------------+----------+----------+
|Female|           Dachun| 25090.869140625| 25090.869140625|  25090.87|  25090.87|
|Female|      Trollhättan|106623.369140625|53311.6845703125|  26830.47|   79792.9|
|  Male|          Wenshao| 18941.509765625| 18941.509765625|  18941.51|  18941.51|
|Female|            Lanas| 13765.900390625| 13765.900390625|   13765.9|   13765.9|
|  Male|            Mörön|    77940.078125|    77940.078125|  77940.08|  77940.08|
|Female|             Same|   73369.7265625|   73369.7265625|  73369.73|  73369.73|
|Female|          Sawahan|  24608.83984375|  24608.83984375|  24608.84|  24608.84|
|  Male|Monte da Boavista|     98586.71875|     98586.71875|  98586.72|  98586.72|
|Female|         Nusajaya|    71637.921875|    71637.921875|  71637.92|  71637.92|
|Fem

## Writing Dataframes to files

In [ ]:
# df3.write.csv('df3.csv')
df3.write.json('df3.json')
df3.write.parquet('df3.parquet')

## working with new data

In [ ]:
df_challenge = spark.read.csv('challenge.csv', header=True)
df_challenge.show()

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

In [ ]:
df_challenge.dtypes

[('ip_address', 'string'),
 ('Country', 'string'),
 ('Domain Name', 'string'),
 ('Bytes_used', 'string')]

In [ ]:
myschema = StructType([
  StructField("ip_address", StringType()),
  StructField("Country", StringType()),
  StructField("Domain Name", StringType()),
  StructField("Bytes_used", IntegerType()
  )])

df_challenge = spark.read.csv('challenge.csv', header=True, schema=myschema)
df_challenge.show()

+---------------+--------------+-----------------+----------+
|     ip_address|       Country|      Domain Name|Bytes_used|
+---------------+--------------+-----------------+----------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|
| 119.239.207.13|         China|         youtu.be|        51|
|  68.69.217.210|         China|        adobe.com|        10|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|
|   211.13.10.68|     Indonesia|          hud.gov|        29|
|   239.80.21.97|      Suriname|       smh.com.au|       218|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|
| 127.242.24.138|         China| surveymonkey.com|       123|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|
|   237.54.11.63|         China|       amazon.com|        83|
| 252.141.157.25|         Japan|      cornell.edu|       374|
|185.220.128.248|       Belgium|       weebly.com|       389|
|   151.77.19.45|   Afghanistan|independent.co.uk|       282|
|  9.161

In [ ]:
df_challenge.dtypes

[('ip_address', 'string'),
 ('Country', 'string'),
 ('Domain Name', 'string'),
 ('Bytes_used', 'int')]

In [ ]:
df_challenge = df_challenge.withColumn('Is_Mexico', when(df_challenge.Country == 'Mexico', 'Yes').otherwise('No'))

In [ ]:
df_challenge.show()

+---------------+--------------+-----------------+----------+---------+
|     ip_address|       Country|      Domain Name|Bytes_used|Is_Mexico|
+---------------+--------------+-----------------+----------+---------+
|  52.81.192.172|         China| odnoklassniki.ru|       463|       No|
| 119.239.207.13|         China|         youtu.be|        51|       No|
|  68.69.217.210|         China|        adobe.com|        10|       No|
|   7.191.21.223|      Bulgaria|     linkedin.com|       853|       No|
|   211.13.10.68|     Indonesia|          hud.gov|        29|       No|
|   239.80.21.97|      Suriname|       smh.com.au|       218|       No|
|106.214.106.233|       Jamaica|    amazonaws.com|        95|       No|
| 127.242.24.138|         China| surveymonkey.com|       123|       No|
|     99.2.6.139|Czech Republic|     geocities.jp|       322|       No|
|   237.54.11.63|         China|       amazon.com|        83|       No|
| 252.141.157.25|         Japan|      cornell.edu|       374|   

In [ ]:
df_challenge.describe().show()

+-------+------------+-----------+-------------+------------------+---------+
|summary|  ip_address|    Country|  Domain Name|        Bytes_used|Is_Mexico|
+-------+------------+-----------+-------------+------------------+---------+
|  count|        1000|       1000|         1000|              1000|     1000|
|   mean|        null|       null|         null|           514.369|     null|
| stddev|        null|       null|         null|287.66708047357156|     null|
|    min|0.23.109.109|Afghanistan|123-reg.co.uk|                 1|       No|
|    max|99.73.193.76|     Zambia|   zimbio.com|              1000|      Yes|
+-------+------------+-----------+-------------+------------------+---------+



In [ ]:
df_challenge.groupBy('Is_Mexico').agg(sum('Bytes_used').alias('total_bytes')).show()

+---------+-----------+
|Is_Mexico|total_bytes|
+---------+-----------+
|       No|     508076|
|      Yes|       6293|
+---------+-----------+



In [ ]:
df_challenge.groupBy('Country').agg(countDistinct('ip_address').alias('unique_ips')).sort(desc('unique_ips')).show()

+--------------+----------+
|       Country|unique_ips|
+--------------+----------+
|         China|       172|
|     Indonesia|       114|
|   Philippines|        65|
|        Russia|        56|
|        Brazil|        35|
|        Poland|        31|
|        Sweden|        28|
|         Japan|        25|
|      Portugal|        23|
|Czech Republic|        23|
|        France|        21|
|          Peru|        19|
|      Colombia|        17|
| United States|        15|
|       Ukraine|        14|
|     Argentina|        14|
|        Mexico|        13|
|      Thailand|        12|
|       Nigeria|        11|
|        Canada|        11|
+--------------+----------+
only showing top 20 rows

